Create database

In [88]:
import pandas as pd
import matplotlib.pyplot as plt

import engarde.decorators as ed

import toolbox.toolbox as tb

pd.set_option('display.max_colwidth', None)


data = 'Schweizer-Nährwertdatenbank-V6.1.xlsx'
foods = tb.read(data)


my_foods = {
    'avocado, roh': 'avocado',
    'blattsalat (durchschnitt), roh': 'lettuce',
    'blumenkohl, gedämpft (ohne zugabe von salz)': 'cauliflower',
    'champignon, roh': 'mushrooms',    
    'fleisch (durchschnitt exkl. innereien), roh': 'meat',
    'gemüse (durchschnitt), roh': 'vegetables',
    'hart- und halbhartkäse, vollfett (durchschnitt)': 'cheese',
    'hühnerei, ganz, festgekocht': 'eggs',
    'kakaopulver': 'cocoa powder', 
    'kochbutter': 'butter',
    'kokosfett': 'coconut fat',
    'kokosnuss, getrocknet (kokosrapseln, kokosflocken)': 'coconut flakes',
    'mandel': 'almond',
    'olivenöl': 'olive oil',
    'peperoni, rot, roh': 'red pepper',
    'poulet, schenkel, mit haut, gebraten (ohne zusatz von fett und salz)': 'chicken leg',
    'poulet, brust, schnitzel oder geschnetzeltes, gebraten (ohne zusatz von fett und salz)': 'chicken breast',
    'samen, kerne, nüsse (durchschnitt), ungesalzen': 'nuts and seed mix',
    'schwein, hals, steak, gebraten (ohne zusatz von fett und salz)': 'pork steak',
    'spinat, roh': 'spinach', 
    'tofu': 'tofu',
    'vollrahm, uht': 'cream'
}

myfoods = tb.make_myfoods(foods, my_foods).set_index('name')

Find items

In [2]:
tb.find_item(foods, 'vollrah')

,name,category,units,calories,fat,carbs,fiber,protein,net_carbs
935,"vollrahm, pasteurisiert",fette und öle/rahm;milch und milchprodukte/rahm und butter,pro 100g essbarer Anteil,334,34.8,3.1,0.0,2.0,3.1
936,"vollrahm, uht",fette und öle/rahm;milch und milchprodukte/rahm und butter,pro 100g essbarer Anteil,335,34.9,3.1,0.0,2.0,3.1


Define meals

In [9]:
def calc_nutrients(data, meal):
    
    nutrients = ['calories', 'fat', 'protein', 'net_carbs']
    
    data = data.loc[data.name.isin(meal), nutrients].sum()
        
    grams = data[1:].copy()
    grams['total'] = grams.sum()
    
    cals = data[1:].copy()
    cals['fat'] = grams.fat * 9
    cals['protein'] = grams.protein * 4
    cals['net_carbs'] = grams.net_carbs * 4
    cals['total'] = data['calories']
    
    nuts = pd.concat([grams, cals], axis=1)
    nuts = nuts.reindex(['total', 'fat', 'protein', 'net_carbs'])
    nuts[nuts < 0] = 0
    nuts.columns = ['grams', 'cals']
    
    return nuts


def make_dashboard(meals):

    def make_doughnut(data, axis, meal_name):
        total_cals = data.loc['total', 'cals']
        nuts = data[1:]
        
        ax[axis].pie(nuts.cals, autopct='%2.1f%%')
        ax[axis].set_title(f"{meal_name.capitalize()} ({total_cals:4.0f} cals)")
        
    fig, ax = plt.subplots(1, len(meals), figsize=(16, 8))

    for meal_num, meal_name in enumerate(meals):
        make_doughnut(meals[meal_name], meal_num, meal_name)

    fig.legend(['fat', 'protein', 'net_carbs'], 
               loc='lower center', 
               borderaxespad=6, ncol=3);
    
meals = {
    
    'breakfast': {
        'eggs': 2,
        'spinach': 1,
        'cauliflower': 1,
        'cheese': 9
    },
    
    'lunch': {
        'lettuce': 2,
        'avocado': 1.5,
        'olive oil': .3,
        'red pepper': 1,
        'nuts and seed mix': .5,
        'tofu': 2,
        'cheese': 4
    },

#     'supper': {
#         'vegetables': 0,
#         'avocado': 1.5,
#         'butter': .2,
#         'pork steak': 0,
#         'cheese': 0
#     }
}
    
meal_nuts = {meal: calc_nutrients(myfoods, meals[meal]) for meal in meals}
meal_nuts['total'] = sum([meal_nuts[meal] for meal in meal_nuts])

# make_dashboard(meal_nuts)

meal_nuts

{'breakfast':            grams   cals
 total       87.6  605.0
 fat         44.2  397.8
 protein     45.0  180.0
 net_carbs    0.0    0.0,
 'lunch':            grams    cals
 total      255.3  2090.0
 fat        192.4  1731.6
 protein     61.2   244.8
 net_carbs    1.7     6.8,
 'total':            grams    cals
 total      342.9  2695.0
 fat        236.6  2129.4
 protein    106.2   424.8
 net_carbs    1.7     6.8}

In [157]:
def calc_nutrients(data, meal):
    
    nutrients = ['calories', 'fat', 'protein', 'net_carbs']
    macros = ['fat', 'protein', 'net_carbs']
    portions = list(meal.values())
    
    data = (data.loc[data.index.isin(meal), nutrients]
            .reindex(meal)
            .multiply(portions, axis=0))
    
    grams = data[macros].copy().T.sum(1)
    grams[grams < 0] = 0
    
    cals = pd.Series(dtype='float')
    cals['fat'] = grams.fat * 9
    cals['protein'] = grams.protein * 4
    cals['net_carbs'] = grams.net_carbs * 4
    cals['total'] = data['calories'].sum()
    cals[cals < 0] = 0
    
#     nuts = pd.concat([grams, cals], axis=1)
#     nuts = nuts.reindex(['total', 'fat', 'protein', 'net_carbs'])
#     nuts[nuts < 0] = 0
#     nuts.columns = ['grams', 'cals']
    
    return ca

meals = {'lunch': {
    'lettuce': 2,
    'avocado': 1.5,
}
}

# ports = list(meals['lunch'].values())

calc_nutrients(myfoods, meals['lunch'])

fat          21.9
protein       5.9
net_carbs     0.0
dtype: float64